# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [1]:
from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import json
import requests
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. 

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [3]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [4]:
# Define the filters we'll use to find our data

item_types = ["REOrthoTile", "PSOrthoTile"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [5]:
indent(combined_filter)

{
  "type": "AndFilter",
  "config": [
    {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.47455596923828,
              37.810326435534755
            ],
            [
              -122.49172210693358,
              37.795406713958236
            ],
            [
              -122.52056121826172,
              37.784282779035216
            ],
            [
              -122.51953124999999,
              37.6971326434885
            ],
            [
              -122.38941192626953,
              37.69441603823106
            ],
            [
              -122.38872528076173,
              37.705010235842614
            ],
            [
              -122.36228942871092,
              37.70935613533687
            ],
            [
              -122.34992980957031,
              37.727280276860036
            ],
            [
              -122.3777

Now let's build the request:

In [6]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [11]:
request

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/f5dfc1298e1249848ac99df4e981c1e3',
  'results': 'https://api.planet.com/data/v1/searches/f5dfc1298e1249848ac99df4e981c1e3/results'},
 'created': '2022-12-09T19:43:27.793325Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    

In [29]:
# Search the Data API
# The default result limit is 100. Here, we're setting our limit to 50
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.run_search(search_id=request['id'], limit=50)
    item_list = [i async for i in items]

Now, we can iterate through our search results.

In [30]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

6128289_1056417_2022-12-07_2262 PSOrthoTile
6117841_1056517_2022-12-02_247f PSOrthoTile
6117841_1056516_2022-12-02_247f PSOrthoTile
6117841_1056417_2022-12-02_247f PSOrthoTile
6117841_1056416_2022-12-02_247f PSOrthoTile
6117149_1056517_2022-12-02_2430 PSOrthoTile
6117149_1056516_2022-12-02_2430 PSOrthoTile
6104170_1056517_2022-11-26_2423 PSOrthoTile
6104170_1056417_2022-11-26_2423 PSOrthoTile
6099543_1056517_2022-11-24_242d PSOrthoTile
6099543_1056417_2022-11-24_242d PSOrthoTile
6095810_1056416_2022-11-22_249a PSOrthoTile
6095810_1056517_2022-11-22_249a PSOrthoTile
6095810_1056516_2022-11-22_249a PSOrthoTile
6095810_1056417_2022-11-22_249a PSOrthoTile
6093412_1056516_2022-11-21_2430 PSOrthoTile
6093412_1056517_2022-11-21_2430 PSOrthoTile
6093412_1056417_2022-11-21_2430 PSOrthoTile
6093412_1056416_2022-11-21_2430 PSOrthoTile
6089131_1056517_2022-11-19_2465 PSOrthoTile
6089131_1056516_2022-11-19_2465 PSOrthoTile
6089131_1056417_2022-11-19_2465 PSOrthoTile
6089131_1056416_2022-11-19_2465 

In [31]:
with open('output/results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads.
Let's start by looking at one item and the assets available to download for that item.

For more information and Assets and Items, check out [Items & Assets](https://developers.planet.com/docs/apis/data/items-assets/) on Planet Developer Center.

In [59]:
# As an example, let's look at the first result in our item_list and grab the item_id and item_type:
item = item_list[0]

print(item['id'], item['properties']['item_type'])

6128289_1056417_2022-12-07_2262 PSOrthoTile


In [48]:
# The list of assets for an item that a user has access to can be retrieved with `permissions`
item['_permissions']

['assets.analytic:download',
 'assets.analytic_5b:download',
 'assets.analytic_5b_xml:download',
 'assets.analytic_dn:download',
 'assets.analytic_dn_xml:download',
 'assets.analytic_sr:download',
 'assets.analytic_xml:download',
 'assets.udm:download',
 'assets.udm2:download',
 'assets.visual:download',
 'assets.visual_xml:download']

In [49]:
# Define our item_id, and item_type

item_id = item['id']
item_type = item['properties']['item_type']

There are a few steps involved in order to download an asset using the Planet Python Client:

* **Get Asset:** Get a desscription of our asset based on the specifications we're looking for
* **Activate Asset:** Activate the asset with the given description
* **Wait Asset:** Wait for the asset to be activated
* **Download Asset:** Now our asset is ready for download!

Let's go through these steps below. We'll do this for our `analytic` asset, as well as the `analytic_xml` asset.

In [50]:
# Analytic Asset
async with Session() as sess:
    cl = DataClient(sess)
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='analytic')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

output/6128289_1056417_2022-12-07_2262_BGRN_Analytic.tif: 100%|█| 416k/416k [02:54<00:00, 2.50M


In [51]:
# Analytic XML Asset
async with Session() as sess:
    cl = DataClient(sess)
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='analytic_xml')
    # Activate Asset
    await cl.activate_asset(asset=asset_desc)
    # Wait Asset (this may take some time!)
    await cl.wait_asset(asset=asset_desc)
    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

output/6128289_1056417_2022-12-07_2262_BGRN_Analytic_metadata.xml: 100%|█| 0.01k/0.01k [00:00<0


Congratulations! Both the `analytic` and `analytic_xml` assets should be saved in our outputs directory.

## Saved Searches

The Planet Python Client can also help in managing saved searches on the Planet Platform.

In [52]:
async with Session() as sess:
    cl = DataClient(sess)
    searches = await cl.list_searches()
    searches_list = [i async for i in searches]

In [53]:
# We may have a lot of saved searches!

len(searches_list)

100

View your saved searches:

In [54]:
for search in searches_list[:100]:
    print(search['id'], search['name'])

f5dfc1298e1249848ac99df4e981c1e3 planet_client_demo
43dca6cff2944a239d96b4b39263c464 43dca6cff2944a239d96b4b39263c464
816d88e6e2ab4719abd0df7fd1403fb7 816d88e6e2ab4719abd0df7fd1403fb7
0e5c694738fa472ca365e91828bbff9c 0e5c694738fa472ca365e91828bbff9c
84fd57c2549d4658bdb60b2478f93528 84fd57c2549d4658bdb60b2478f93528
682f64d5f0304ea08103689919740354 682f64d5f0304ea08103689919740354
22568456f1cc4c69a80cb0121b1271bf 22568456f1cc4c69a80cb0121b1271bf
1dbd454ee3c54ff4ae457a8f136fd642 1dbd454ee3c54ff4ae457a8f136fd642
fb73db3ee010410e81cdbd39c1355ce1 fb73db3ee010410e81cdbd39c1355ce1
0f1d6706b8d14f7896477a6df6409c47 0f1d6706b8d14f7896477a6df6409c47
aff65930c69b4c818c778e2e87ae4b84 aff65930c69b4c818c778e2e87ae4b84
d5e28fe0154944d98317cc19fff4c594 d5e28fe0154944d98317cc19fff4c594
4a01df80dce443199e902b023e30b17c 4a01df80dce443199e902b023e30b17c
6df9c275d6a2448c821f9d395a91ce2e 6df9c275d6a2448c821f9d395a91ce2e
f0dd5a03dd5b48d0ad8538fd0d044c51 f0dd5a03dd5b48d0ad8538fd0d044c51
14f0f4bdf11a41658510a5b8

We can find the saved search we're looking for by filtering on our search id:

In [55]:
async with Session() as sess:
    cl = DataClient(sess)
    search = await cl.get_search(searches_list[0]['id'])

In [56]:
search

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/f5dfc1298e1249848ac99df4e981c1e3',
  'results': 'https://api.planet.com/data/v1/searches/f5dfc1298e1249848ac99df4e981c1e3/results'},
 'created': '2022-12-09T19:43:27.793325Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    

# Statistics

The Python API client can also help report statistical summaries of the amount of data in the Planet API.

In [57]:
# Here, we can look at the stats for the search we've been working with:

async with Session() as sess:
    cl = DataClient(sess)
    stats = await cl.get_stats(interval='year', search_filter=combined_filter, item_types=item_types)

In [58]:
indent(stats)

{
  "buckets": [
    {
      "count": 375,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 594,
      "start_time": "2019-01-01T00:00:00.000000Z"
    },
    {
      "count": 665,
      "start_time": "2020-01-01T00:00:00.000000Z"
    },
    {
      "count": 758,
      "start_time": "2021-01-01T00:00:00.000000Z"
    },
    {
      "count": 588,
      "start_time": "2022-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}
